# Algoritmos de multiplicação

Aprendemos na escola a multiplicar duas matrizes $A \in \mathbb{R}^{m \times p}$ e $B \in \mathbb{R}^{p \times n}$ usando a fórmula $c_{i, j} = \sum \limits_{k = 1}^p a_{i, k} b_{k, j}$. O que não nos ensinam é que há mais duas formas de interpretar a multiplicação.

## Interpretação 1: produto escalar

Forma mais tradicional. Basta observarmos que se $C = A \cdot B$, então
$$
c_{i, j} = \sum \limits_{k = 1}^p a_{i, k} b_{k, j} = A_i \odot B^j
$$

In [1]:
A = [1 2 3 4;
     5 6 7 8];

B = [1 2 1;
     3 4 1;
     5 6 1;
     7 8 1];

In [2]:
# Comandos para obter as dimensoes das matrizes
mA, nA = size(A)
mB, nB = size(B)

println("Dimensao A: $(mA) x $(nA).")
println("Dimensao B: $(mB) x $(nB).")

m = mA
p = nA # ou mB
n = nB

# Escolhe i e j
i = 2
j = 3

# Calcula o somatorio
cij = 0

for k = 1:p
    cij = cij + A[i, k] * B[k, j]
end

println("O elemento C_$(i),$(j) = $(cij).")

Dimensao A: 2 x 4.
Dimensao B: 4 x 3.
O elemento C_2,3 = 26.


Experimente:

  - Modificar os valores das variáveis `i` e `j`
  - Colocar valores de `i` e `j` maiores que a dimensão das matrizes
  - Definir `p` como `mB` ao invés de `nA`
  
  Podemos também substituir as linhas 17-21 do somatório pela função `dot` do Julia que calcula o produto escalar

In [3]:
cij = dot(A[i,:], B[:, j])

println("O elemento C_$(i),$(j) = $(cij).")

O elemento C_2,3 = 26.


Os elementos $c_{i, j}$ têm que ser calculados para todo $i$ e $j$, e para isso usamos 2 laços do tipo `for`

In [4]:
C = Array(Float64, m, n)

for i = 1:m
    for j = 1:n
        
        # Aqui copiamos o codigo de cij
        # Calcula o somatorio
        C[i, j] = 0

        for k = 1:p
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        
    end
end

display(C)

2×3 Array{Float64,2}:
  50.0   60.0  10.0
 114.0  140.0  26.0

Experimente trocar as ordem dos laços `for` nas linhas 3 e 4. A única diferença é na forma como calculamos os elementos de $C$.

A função abaixo implementa a multiplicação usando essa ideia.

In [5]:
function multiplica_mat_1(A, B)
    
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB
    
    C = Array(Float64, m, n)

    for i = 1:m
        for j = 1:n

            # Aqui copiamos o codigo de cij
            # Calcula o somatorio
            C[i, j] = 0

            for k = 1:p
                C[i, j] = C[i, j] + A[i, k] * B[k, j]
            end

        end
    end
    
    return C
    
end

multiplica_mat_1 (generic function with 1 method)

In [6]:
multiplica_mat_1(A, B)

2×3 Array{Float64,2}:
  50.0   60.0  10.0
 114.0  140.0  26.0

Experimentos com matrizes aleatórias grandes.

In [8]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_1(A1, B1)))

Tempo para  100: 0.001497347
Tempo para  200: 0.012077685
Tempo para  400: 0.023268368
Tempo para  800: 0.92554369
Tempo para 1000: 1.18009939


Observe que quando **dobramos** a dimensão das matrizes, o tempo é **multiplicado por 8**. Dizemos que o custo computacional da multiplicação tradicional é $O(n^3)$, onde $n$ é a dimensão da matriz.

## Interpretação 2: por linha ou colunas

A segunda interpretação ocorre quando olhamos a forma de uma coluna ou linha de $C$: $C_i$ ou $C^j$.
$$
C = A \cdot B = 
\begin{bmatrix}
A \cdot B^1 & \cdots & A \cdot B^n
\end{bmatrix}
=
\begin{bmatrix}
A_1 \cdot B\\ \vdots\\ A_m \cdot B
\end{bmatrix}
$$
onde $A \cdot B^j$ representa o produto entre a matriz $A$ e a coluna $j$ de $B$, e $A_i \cdot B$ é o produto entre a linha $i$ de $A$ e a matriz $B$ inteira.

### Calculando $C^j$

In [2]:
A = [1 2 3 4;
     5 6 7 8];

B = [1 2 1;
     3 4 1;
     5 6 1;
     7 8 1];

In [27]:
# Comandos para obter as dimensoes das matrizes
mA, nA = size(A)
mB, nB = size(B)

println("Dimensao A: $(mA) x $(nA).")
println("Dimensao B: $(mB) x $(nB).")

m = mA
p = nA # ou mB
n = nB

# Construindo a coluna j.
# Precisa colocar zeros para somar.
cj = zeros(Float64, m)

j = 3

for i = 1:m
    
    for k = 1:p
    
        cj[i] = cj[i] + A[i, k] * B[k, j]
        
    end
    
end

println("C^$(j) = ", cj)

Dimensao A: 2 x 4.
Dimensao B: 4 x 3.
C^3 = [10.0,26.0]


As duas versões desse código são apresentadas abaixo.

In [9]:
"""
Esta funcao multiplica duas matrizes, construindo a matriz produto por **colunas**.
"""
function multiplica_mat_2_col_ik(A, B)
    
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB
    
    C = zeros(Float64, m, n)
    
    for j = 1:n
        
        for i = 1:m

            for k = 1:p
                
                # Atencao aqui! Temos que trocar cj por C[:, j]!
                # Ou seja, a j-esima coluna de C
                C[i, j] = C[i, j] + A[i, k] * B[k, j]

            end

        end
        
    end
    
    return C
    
end

"""
Esta funcao multiplica duas matrizes, construindo a matriz produto por **colunas**.

Nesta versao, os lacos associados com `i` e `k` foram trocados de ordem: primeiro `k` e
depois `i`.
"""
function multiplica_mat_2_col_ki(A, B)
    
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB
    
    C = zeros(Float64, m, n)
    
    for j = 1:n
        
        for k = 1:p

            for i = 1:m
                
                # Atencao aqui! Temos que trocar cj por C[:, j]!
                # Ou seja, a j-esima coluna de C
                C[i, j] = C[i, j] + A[i, k] * B[k, j]

            end

        end
        
    end
    
    return C
    
end

multiplica_mat_2_col_ki

In [10]:
multiplica_mat_2_col_ik(A, B)

2×3 Array{Float64,2}:
  50.0   60.0  10.0
 114.0  140.0  26.0

In [11]:
multiplica_mat_2_col_ki(A, B)

2×3 Array{Float64,2}:
  50.0   60.0  10.0
 114.0  140.0  26.0

In [12]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

Tempo para  100: 0.021346191
Tempo para  200: 0.021350887
Tempo para  400: 0.174076294
Tempo para  800: 1.894931158
Tempo para 1000: 3.504051009


Experimentos para quando trocamos os laços `for` de ordem

In [13]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

Tempo para  100: 0.023084222
Tempo para  200: 0.010044288
Tempo para  400: 0.069490005
Tempo para  800: 0.604068085
Tempo para 1000: 1.021626343


**Por que** a segunda função foi duas vezes mais rápida que a primeira?

### Calculando $C_i$

In [25]:
A = [1 2 3 4;
     5 6 7 8];

B = [1 2 1;
     3 4 1;
     5 6 1;
     7 8 1];

In [26]:
# Comandos para obter as dimensoes das matrizes
mA, nA = size(A)
mB, nB = size(B)

println("Dimensao A: $(mA) x $(nA).")
println("Dimensao B: $(mB) x $(nB).")

m = mA
p = nA # ou mB
n = nB

# Construindo a linha i.
# Precisa colocar zeros para somar.
ci = zeros(Float64, n)

i = 2

for j = 1:n
    
    for k = 1:p
    
        ci[j] = ci[j] + A[i, k] * B[k, j]
        
    end
    
end

println("C_$(i) = ", ci)

Dimensao A: 2 x 4.
Dimensao B: 4 x 3.
C_2 = [114.0,140.0,26.0]


## Usando funções já implementadas

In [22]:
using Base.LinAlg.BLAS

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)
println(@elapsed(gemm('N', 'N', 1.0, A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)
println(@elapsed(A1 * B1))

0.038054193
0.024211019
